In [1]:
%load_ext autoreload
%reload_ext tensorboard
%autoreload 2

In [2]:
from random import shuffle
from ssl import SSLSession
from tokenize import Whitespace
# from model_transformer import build_transformer
from dataset import BilingualDataset, causal_mask
from config import get_config, get_weights_file_path

import torchtext.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import LambdaLR

import warnings
from tqdm import tqdm
import os
from pathlib import Path

# Huggingface datasets and tokenizers
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

import torchmetrics
# Launch TensorBoard SSLSession
from torch.utils.tensorboard import SummaryWriter

/home/sn/anaconda3/envs/fastai2022/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
from config import get_config
import torch
import torch.optim as optim

from model_transformer import transformerModel, saveCallback #, PrintCallback #YOLOv3
from train import greedy_decode, get_ds
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
from pytorch_lightning import LightningModule, Trainer

In [4]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
AVAIL_GPUS

1

In [5]:
from pytorch_lightning.callbacks import ModelSummary, RichProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor
lr_monitor = LearningRateMonitor(logging_interval='step')

num_examples = 2

cfg = get_config()
cfg['batch_size'] = 8
cfg['preload'] = None
cfg['num_epochs'] = 10

# Make sure the weights folder exists
Path(cfg['model_folder']).mkdir(parents=True, exist_ok=True)

train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(cfg)

#Tensorboard
writer = SummaryWriter(cfg['experiment_name'])

model = transformerModel(cfg, tokenizer_src, tokenizer_tgt, writer, num_examples)

trainer = Trainer(
    callbacks=[ModelSummary(max_depth=-1), lr_monitor], #saveCallback()], # PrintCallback()], RichProgressBar(leave=True)
#     default_root_dir="/home/sn/ERAv1/checkpoints/",
    enable_checkpointing=True,
    precision=16,
    devices=AVAIL_GPUS, 
    max_epochs=cfg['num_epochs'],
    # limit_train_batches = 10,
    limit_val_batches = num_examples,
    log_every_n_steps=500
) 

Max Length of source sentence: 471
Max Length of target sentence: 482


Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
#             ckpt_path="/ERAv1/session15_assignment/weights/s15Model.ckpt")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loading `train_dataloader` to estimate number of stepping batches.

    | Name                                                  | Type                    | Params
----------------------------------------------------------------------------------------------------
0   | model                                                 | Transformer             | 68.1 M
1   | model.encoder                                         | Encoder                 | 9.4 M 
2   | model.encoder.layers                                  | ModuleList              | 9.4 M 
3   | model.encoder.layers.0                                | EncoderBlock            | 3.1 M 
4   | model.encoder.layers.0.self_attention_block           | MultiHeadAttentionBlock | 1.0 M 
5   | model.encoder.layers.0.self_attention_block.w_q       | Linear                  | 262 K 
6   | model.encoder.layers.0.self_attention_block.w_k       | Linear                  | 262 K 
7   | model.encoder.layers.

Sanity Checking: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: Then d’Artagnan ceased knocking, and prayed with an accent so full of anxiety and promises, terror and cajolery, that his voice was of a nature to reassure the most fearful.
    TARGET: Alors d'Artagnan cessa de frapper et pria, avec un accent si plein d'inquiétude et de promesses, d'effroi et de cajolerie, que sa voix était de nature à rassurer de plus peureux.
 PREDICTED: Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours Cours
--------------------------------------------------------------------------------
    SOURCE: Le but de la duchesse, que toutefois elle ne s’avouait pas, était de presser le mariage du marquis Crescenzi : Fabrice, de son côté, fit la route dans des transports de bonheur fous, et qui semblèrent ridicules à sa

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: "Ayrton!
    TARGET: -- Ayrton!
 PREDICTED: 
--------------------------------------------------------------------------------
    SOURCE: "That is strong," she said, when she had finished: "I relish it."
    TARGET: «C'est fort, dit-elle, lorsqu'elle eut fini; j'aime cela.»
 PREDICTED: 
--------------------------------------------------------------------------------


Validation: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: The king took it with a trembling hand, looked for the address, which was wanting, became very pale, opened it slowly, then seeing by the first words that it was addressed to the King of Spain, he read it rapidly.
    TARGET: Le roi la prit d'une main tremblante, chercha l'adresse, qui manquait, devint très pâle, l'ouvrit lentement, puis, voyant par les premiers mots qu'elle était adressée au roi d'Espagne, il lut très rapidement.
 PREDICTED: 
--------------------------------------------------------------------------------
    SOURCE: He pressed a metal button and at once the propeller slowed down significantly.
    TARGET: Il pressa un bouton de métal, et aussitôt la vitesse de l'hélice fut très diminuée.
 PREDICTED: 
--------------------------------------------------------------------------------


`Trainer.fit` stopped: `max_epochs=10` reached.


In [7]:
# start tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
